In [ ]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

df = pd.read_csv("updatedInfo.csv")

df.head

Saving updatedInfo.csv to updatedInfo.csv


<bound method NDFrame.head of          ID  GrossMonthlyIncome  CreditCardPayment  CarPayment  \
0         1                3103                317         374   
1         2                2939                368         395   
2         3                4676                222         352   
3         4                8469                224         416   
4         5                2406                497         385   
...     ...                 ...                ...         ...   
9995   9996                4329                483         473   
9996   9997                7037                465         408   
9997   9998                9086                284         352   
9998   9999                9655                443         365   
9999  10000                4086                457         373   

      StudentLoanPayments  AppraisedValue  DownPayment  LoanAmount  \
0                     250          268468     32216.16   236251.84   
1                     228          33

In [ ]:
df = df.drop(["ID"], axis=1)
df = df.dropna()

In [ ]:
df['creditScoreGood'] = (df['CreditScore'] >= 640).astype(int)
df['LTVGood'] = (df['LTV'] < 95).astype(int)
df['FEDTIGood'] = (df['FEDTI'] <= 28).astype(int)
df["DTIGood"] = (df['DTI'] <= 36).astype(int)
df['Result'] = (df[['creditScoreGood', 'FEDTIGood', 'DTIGood', 'LTVGood']] == 1).all(axis=1).astype(int)
columns_to_drop = ["creditScoreGood", "FEDTIGood", "DTIGood", 'LTVGood']
df = df.drop(columns=columns_to_drop)

print(df)

      GrossMonthlyIncome  CreditCardPayment  CarPayment  StudentLoanPayments  \
0                   3103                317         374                  250   
1                   2939                368         395                  228   
2                   4676                222         352                  368   
3                   8469                224         416                  400   
4                   2406                497         385                  444   
...                  ...                ...         ...                  ...   
9995                4329                483         473                  306   
9996                7037                465         408                  338   
9997                9086                284         352                  288   
9998                9655                443         365                  407   
9999                4086                457         373                  392   

      AppraisedValue  DownPayment  Loan

In [ ]:
num_entries_with_result_1 = df['Result'].sum()
print("Number of entries where Result is 1:", num_entries_with_result_1)

num_entries_with_result_0 = (df['Result'] == 0).sum()
print("Number of entries where Result is 0:", num_entries_with_result_0)

Number of entries where Result is 1: 1869
Number of entries where Result is 0: 8131


In [ ]:
X = df.drop("Result", axis=1)
y = df["Result"]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
acc_log = round(logreg.score(X_test, y_test) * 100, 2)
print("Test Accuracy: ", acc_log)

Test Accuracy:  93.55


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import numpy as np

default_grossMonthlyIncome = np.nan
default_creditCardPayment = np.nan
default_carPayment = np.nan
default_studentLoanPayments = np.nan
default_appraisedValue = np.nan
default_downPayment = np.nan
default_loanAmount = np.nan
default_monthlyMortgagePayment = np.nan
default_creditScore = np.nan

specific_person_data = [default_grossMonthlyIncome, default_creditCardPayment, default_carPayment, default_studentLoanPayments, default_appraisedValue,
                        default_downPayment, default_loanAmount, default_monthlyMortgagePayment, default_creditScore]

actual_loanAmount = 50000  # Replace with the actual value
actual_creditScore = 700  # Replace with the actual value
actual_creditCardPayment = 100  # Replace with the actual value

specific_person_data[6] = actual_loanAmount  # Index 6 corresponds to loanAmount
specific_person_data[8] = actual_creditScore  # Index 8 corresponds to creditScore
specific_person_data[1] = actual_creditCardPayment  # Index 1 corresponds to creditCardPayment

specific_person_prediction = logreg.predict([specific_person_data])

if specific_person_prediction == 1:
    print("Loan approved for the specific person.")
else:
    print("Loan not approved for the specific person.")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


ValueError: ignored

In [ ]:
coeff_df = pd.DataFrame(df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
0,CreditCardPayment,0.001826
6,MonthlyMortgagePayment,0.000019
4,DownPayment,0.000016
5,LoanAmount,-0.000003
11,Result,-0.000460
9,DTI,-0.000874
10,FEDTI,-0.000905
8,LTV,-0.003547
3,AppraisedValue,-0.004235
2,StudentLoanPayments,-0.004753
